In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import threading

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Basic Python packages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time


In [2]:
camera = Camera()

In [3]:
# コントローラの作成
controller = widgets.Controller(index=0)
display(controller)



Controller()

In [4]:
def xy_uuid(x, y, r):
    return 'xy_%03d_%03d_%03d_%s' % (x * widget_width / 2 + widget_width / 2, y * widget_height / 2 + widget_height / 2,r * widget_height / 4 + widget_height / 4, uuid1())

def save_snapshot(change):
    if change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value,r_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        free_count.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))


In [10]:
widget_width = camera.width
widget_height = camera.height

image_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)
target_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)

# スライダーの作成
x_slider = widgets.FloatSlider(description='X', min=-1.0, max=1.0, value=0,step = 0.01)
y_slider = widgets.FloatSlider(description='Y', min=-1.0, max=1.0, value=0,step = 0.01)
r_slider = widgets.FloatSlider(description='R', min=-1.0, max=1.0, value=0,step = 0.01)

stick_input_x_slider = widgets.FloatSlider(description='Stick Input X', min=-1.0, max=1.0, value=0,step=0.01)
stick_input_y_slider = widgets.FloatSlider(description='Stick Input Y', min=-1.0, max=1.0, value=0,step=0.01)
stick_input_r_slider = widgets.FloatSlider(description='Stick Input R', min=-1.0, max=1.0, value=0,step=0.01)

dead_zone_slider = widgets.FloatSlider(description='Dead Zone', min=0.0, max=1.0, value=0.1,step=0.01)
step_slider = widgets.FloatSlider(description='Step Size', min=0.001, max=1.000, value=0.1,step = 0.0001)



def display_xy(camera_image):
    image = np.copy(camera_image)
    
    pos_x = int(x_slider.value * widget_width / 2 + widget_width / 2)
    pos_y = int(y_slider.value * widget_height / 2 + widget_height / 2)
    pos_r = int(r_slider.value * widget_width / 4 + widget_width / 4)
    
    if pos_r <= 2:
        pos_r = 2
    
    image = cv2.circle(image, (pos_x, pos_y), pos_r, (0, 255, 0),2)
    jpeg_image = bgr8_to_jpeg(image)
    
    return jpeg_image


# X座標を更新する関数
def update_x():
    dead_zone = dead_zone_slider.value
    step = step_slider.value
    x_change = 0
    y_change = 0
    r_change = 0

    if abs(stick_input_x_slider.value) > dead_zone:
        x_change = stick_input_x_slider.value * step
        
    if abs(stick_input_y_slider.value) > dead_zone:
        y_change = stick_input_y_slider.value * step
        
    if abs(stick_input_r_slider.value) > dead_zone:
        r_change = stick_input_r_slider.value * step

    x_slider.value = max(min(x_slider.max, x_slider.value + x_change), x_slider.min)
    y_slider.value = max(min(y_slider.max, y_slider.value + y_change), y_slider.min)
    r_slider.value = max(min(r_slider.max, r_slider.value + r_change), r_slider.min)

    # 定期的にこの関数を再度呼び出す
    threading.Timer(0.05, update_x).start()

    
controller.buttons[5].observe(save_snapshot, names='value')

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

# 最初の更新を開始
update_x()

option = widgets.VBox([dead_zone_slider, step_slider])
position_x = widgets.VBox([x_slider, stick_input_x_slider])
position_y = widgets.VBox([y_slider, stick_input_y_slider])
position_r = widgets.VBox([r_slider, stick_input_r_slider])




In [11]:
img_box = widgets.HBox([image_widget, target_widget])

In [12]:
# スティック入力をスライダーにリンク
widgets.jsdlink((controller.axes[0], 'value'), (stick_input_x_slider, 'value'))
widgets.jsdlink((controller.axes[1], 'value'), (stick_input_y_slider, 'value'))
widgets.jsdlink((controller.axes[3], 'value'), (stick_input_r_slider, 'value'))

DirectionalLink(source=(Axis(value=-1.5259021896696368e-05), 'value'), target=(FloatSlider(value=0.0, descript…

In [13]:

display(widgets.HBox([option,position_x,position_y,position_r]))

In [14]:
display(img_box)

In [ ]:
#camera.stop()